# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

In [8]:
# First, we will need to complete a left-join with Citation and Patent data on the Citing patent number.
# Then, we select the Cited, Citing, and the citing state (POSTATE) columns from the Citing_State dataframe.

Citing_State = citations.join(patents, citations["CITING"] == patents["PATENT"], how = "left").cache()
Citing_State = Citing_State.select("CITED", "CITING", col("POSTATE").alias("CITING_STATE"))

Citing_State.show()

+-------+-------+------------+
|  CITED| CITING|CITING_STATE|
+-------+-------+------------+
|1331793|3858258|          CA|
|1540798|3858258|          CA|
| 924225|3858527|        null|
|2444326|3858527|        null|
|2705120|3858527|        null|
|2967080|3858527|        null|
|3602157|3858527|        null|
|3638586|3858527|        null|
|3699902|3858527|        null|
| 957631|3858560|          IN|
|3675252|3858597|          MT|
|3815160|3858597|          MT|
|2290722|3858770|          CA|
|2777621|3858770|          CA|
|2782969|3858770|          CA|
|3040941|3858770|          CA|
| 982044|3859029|          NY|
|1020004|3859029|          NY|
|1830227|3859029|          NY|
|2752631|3859029|          NY|
+-------+-------+------------+
only showing top 20 rows



In [9]:
# Next we will left-join the Citing_State data and the Patent data on the Cited patent number.
# Then, we select Citing, Citing_State, Cited, and the citing state (POSTATE) columns to form the tempStep dataframe

tempStep = Citing_State.join(patents, Citing_State["CITED"] == patents["PATENT"], how = "left").cache()
tempStep = tempStep.select("CITING", "CITING_STATE", "CITED", col("POSTATE").alias("CITED_STATE"))

tempStep.show()

+-------+------------+-----+-----------+
| CITING|CITING_STATE|CITED|CITED_STATE|
+-------+------------+-----+-----------+
|4305315|          MN| 2366|       null|
|4192521|        null| 2366|       null|
|4253355|          MN| 2366|       null|
|5580635|          WI| 5156|       null|
|4976561|        null| 5518|       null|
|4480374|          MN| 5803|       null|
|5123817|        null| 6620|       null|
|4115020|        null| 7240|       null|
|4727698|          CA| 7253|       null|
|4360982|          IA| 7340|       null|
|4108250|          IL| 7340|       null|
|5692807|          PA|10817|       null|
|5581904|        null|11458|       null|
|4282613|          MI|12940|       null|
|4741426|          NY|13840|       null|
|4705153|          NY|13840|       null|
|4556218|          FL|14832|       null|
|4896714|        null|15447|       null|
|5065652|          OH|15790|       null|
|5058476|          OH|15790|       null|
+-------+------------+-----+-----------+
only showing top

In [10]:
# Since we now have the needed dataframe, we need to scrub the data to remove all null values and filter the data based on matching states.

tempStep = tempStep.filter(col("CITING_STATE").isNotNull() & col("CITED_STATE").isNotNull())
tempStep = tempStep.filter(col("CITING_STATE") == col("CITED_STATE"))

tempStep.show()

+-------+------------+-------+-----------+
| CITING|CITING_STATE|  CITED|CITED_STATE|
+-------+------------+-------+-----------+
|3861359|          IL|3072100|         IL|
|3917094|          WI|3072274|         WI|
|4051847|          CA|3077191|         CA|
|4041217|          MD|3079454|         MD|
|4385248|          NY|3079519|         NY|
|4945561|          NY|3081464|         NY|
|4884717|          MI|3086674|         MI|
|4053105|          MA|3087676|         MA|
|3893618|          MA|3087676|         MA|
|4249365|          IA|3088262|         IA|
|5247786|          IA|3088262|         IA|
|5035582|          PA|3089008|         PA|
|3884773|          NJ|3089888|         NJ|
|3935741|          TX|3090232|         TX|
|4828608|          NY|3093475|         NY|
|5173632|          NH|3094640|         NH|
|4650499|          OK|3097519|         OK|
|4553985|          OK|3097519|         OK|
|4684473|          NJ|3102098|         NJ|
|5956831|          CA|3102333|         CA|
+-------+--

In [11]:
# We now have our scrubed and filtered data, we proceed to group the data on Citing patent number and count the number of same-state citations.

Citing_Count = tempStep.groupby("CITING").count()
Citing_Count.show()

+-------+-----+
| CITING|count|
+-------+-----+
|5300411|    7|
|3956677|    1|
|4171110|    4|
|4031936|    2|
|3894399|    3|
|4761449|    1|
|4369612|    3|
|4052003|    4|
|4151539|    2|
|4414386|    1|
|4868906|    2|
|4339036|    2|
|4668726|   13|
|5279306|    7|
|4608206|    5|
|4673168|    2|
|4923281|    8|
|5912450|   15|
|4559622|    7|
|4841690|    4|
+-------+-----+
only showing top 20 rows



In [12]:
# Finally, we left-join Citing patents and Citing_Count on Patent in the original dataframe, 
# replace Nan/null values with 0 for clarity of 0 count, 
# and order the final output in descending order by number of same-state citations.

finalStep = patents.join(Citing_Count, patents["PATENT"] == Citing_Count["CITING"], how = "left").cache()
finalStep.orderBy(col("count"), ascending = False).fillna(0, ["count"]).show()

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD| CITING|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+-----+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|5959466|  125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|5983822|  103|
|6008204| 1999|